# Hospital & Geographic Analytics

**Objective**: Analyze CMS hospital data for geographic patterns and provider performance.

This notebook processes CMS Hospital Readmissions Reduction Program data to create state-level summaries and hospital metrics for the geographic analysis dashboard.

In [ ]:
# Cell 1: Setup
import pandas as pd
import numpy as np
import json
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Set up paths
DATA_DIR = Path('../data/raw')
OUTPUT_DIR = Path('../data/processed')
OUTPUT_DIR.mkdir(exist_ok=True)

print("Libraries loaded successfully")

In [ ]:
# Cell 2: Load CMS Hospital Readmissions Data
df_hosp = pd.read_csv(DATA_DIR / 'hospital_readmissions.csv')

print("=" * 60)
print("CMS HOSPITAL READMISSIONS DATA")
print("=" * 60)
print(f"\nDataset shape: {df_hosp.shape}")
print(f"\nColumn names:")
for i, col in enumerate(df_hosp.columns):
    print(f"  {i+1}. {col}")
print(f"\nFirst 3 rows:")
df_hosp.head(3)

In [ ]:
# Cell 3: Identify and Map Column Names
# The CMS data columns vary by download - let's detect what we have

print("\n" + "=" * 60)
print("COLUMN DETECTION")
print("=" * 60)

# Common column name variations in CMS data
column_mappings = {
    # Hospital name variations
    'hospital_name': ['Facility Name', 'Hospital Name', 'facility_name', 'hospital_name', 'Provider Name'],
    # State variations
    'state': ['State', 'state', 'Provider State', 'provider_state'],
    # City variations
    'city': ['City', 'city', 'Provider City', 'provider_city'],
    # Rate/score variations
    'readmission_rate': ['Score', 'Excess Readmission Ratio', 'Expected Readmission Rate', 
                         'excess_readmission_ratio', 'Predicted Readmission Rate'],
    # Penalty variations
    'penalty_pct': ['Payment Reduction Percentage', 'payment_reduction_percentage', 
                    'FY 2025 Payment Reduction Percentage'],
    # Patient count variations
    'patient_count': ['Number of Patients', 'number_of_patients', 'Discharges']
}

# Detect actual columns
detected_columns = {}
for target, possible_names in column_mappings.items():
    for name in possible_names:
        if name in df_hosp.columns:
            detected_columns[target] = name
            break

print("\nDetected column mappings:")
for target, actual in detected_columns.items():
    print(f"  {target} -> '{actual}'")

# Create standardized column names
for target, actual in detected_columns.items():
    if actual in df_hosp.columns:
        df_hosp[target] = df_hosp[actual]

In [ ]:
# Cell 4: Data Cleaning
print("\n" + "=" * 60)
print("DATA CLEANING")
print("=" * 60)

# Convert numeric columns (handle 'Not Available' type values)
if 'readmission_rate' in df_hosp.columns:
    df_hosp['readmission_rate'] = pd.to_numeric(df_hosp['readmission_rate'], errors='coerce')
    print(f"readmission_rate: {df_hosp['readmission_rate'].notna().sum()} valid values")

if 'penalty_pct' in df_hosp.columns:
    df_hosp['penalty_pct'] = pd.to_numeric(df_hosp['penalty_pct'], errors='coerce')
    print(f"penalty_pct: {df_hosp['penalty_pct'].notna().sum()} valid values")

if 'patient_count' in df_hosp.columns:
    df_hosp['patient_count'] = pd.to_numeric(df_hosp['patient_count'], errors='coerce')

# Drop rows without state
if 'state' in df_hosp.columns:
    df_hosp = df_hosp.dropna(subset=['state'])
    print(f"\nHospitals with valid state: {len(df_hosp)}")

# For the rate column, if we have excess ratio instead of rate, convert it
# Excess ratio of 1.0 = expected rate, >1.0 = higher than expected
if 'readmission_rate' in df_hosp.columns:
    rate_stats = df_hosp['readmission_rate'].describe()
    print(f"\nReadmission rate statistics:")
    print(rate_stats)
    
    # If values are around 1.0, it's likely an excess ratio - convert to percentage
    if rate_stats['mean'] < 5:
        print("\nDetected excess ratio format - converting to percentage")
        df_hosp['readmission_rate'] = df_hosp['readmission_rate'] * 100

In [ ]:
# Cell 5: State-Level Aggregation
print("\n" + "=" * 60)
print("STATE-LEVEL AGGREGATION")
print("=" * 60)

# Determine which columns we have for aggregation
agg_dict = {'hospital_name': 'count'}  # Count hospitals per state

if 'readmission_rate' in df_hosp.columns and df_hosp['readmission_rate'].notna().any():
    agg_dict['readmission_rate'] = 'mean'
    rate_col = 'readmission_rate'
else:
    rate_col = None

if 'penalty_pct' in df_hosp.columns and df_hosp['penalty_pct'].notna().any():
    agg_dict['penalty_pct'] = 'mean'

# Aggregate by state
state_summary = df_hosp.groupby('state').agg(agg_dict).reset_index()

# Rename columns for clarity
state_summary.columns = ['state'] + [
    'hospital_count' if c == 'hospital_name' else 
    'avg_readmission_rate' if c == 'readmission_rate' else 
    'avg_penalty_pct' if c == 'penalty_pct' else c
    for c in state_summary.columns[1:]
]

# Calculate estimated total penalty (using national average if available)
AVG_MEDICARE_PAYMENTS_PER_HOSPITAL = 5000000  # $5M example
if 'avg_penalty_pct' in state_summary.columns:
    state_summary['total_penalty_estimate'] = (
        state_summary['hospital_count'] * 
        AVG_MEDICARE_PAYMENTS_PER_HOSPITAL * 
        state_summary['avg_penalty_pct'] / 100
    )
else:
    state_summary['total_penalty_estimate'] = 0

# Sort by readmission rate if available
if 'avg_readmission_rate' in state_summary.columns:
    state_summary = state_summary.sort_values('avg_readmission_rate', ascending=False)

print(f"\nTop 10 states by readmission rate:")
print(state_summary.head(10).to_string(index=False))

print(f"\nBottom 10 states by readmission rate:")
print(state_summary.tail(10).to_string(index=False))

In [ ]:
# Cell 6: State Coordinates for Map
# US State coordinates for map visualization
STATE_COORDS = {
    'AL': {'lat': 32.806671, 'lng': -86.791130, 'name': 'Alabama'},
    'AK': {'lat': 61.370716, 'lng': -152.404419, 'name': 'Alaska'},
    'AZ': {'lat': 33.729759, 'lng': -111.431221, 'name': 'Arizona'},
    'AR': {'lat': 34.969704, 'lng': -92.373123, 'name': 'Arkansas'},
    'CA': {'lat': 36.116203, 'lng': -119.681564, 'name': 'California'},
    'CO': {'lat': 39.059811, 'lng': -105.311104, 'name': 'Colorado'},
    'CT': {'lat': 41.597782, 'lng': -72.755371, 'name': 'Connecticut'},
    'DE': {'lat': 39.318523, 'lng': -75.507141, 'name': 'Delaware'},
    'FL': {'lat': 27.766279, 'lng': -81.686783, 'name': 'Florida'},
    'GA': {'lat': 33.040619, 'lng': -83.643074, 'name': 'Georgia'},
    'HI': {'lat': 21.094318, 'lng': -157.498337, 'name': 'Hawaii'},
    'ID': {'lat': 44.240459, 'lng': -114.478828, 'name': 'Idaho'},
    'IL': {'lat': 40.349457, 'lng': -88.986137, 'name': 'Illinois'},
    'IN': {'lat': 39.849426, 'lng': -86.258278, 'name': 'Indiana'},
    'IA': {'lat': 42.011539, 'lng': -93.210526, 'name': 'Iowa'},
    'KS': {'lat': 38.526600, 'lng': -96.726486, 'name': 'Kansas'},
    'KY': {'lat': 37.668140, 'lng': -84.670067, 'name': 'Kentucky'},
    'LA': {'lat': 31.169546, 'lng': -91.867805, 'name': 'Louisiana'},
    'ME': {'lat': 44.693947, 'lng': -69.381927, 'name': 'Maine'},
    'MD': {'lat': 39.063946, 'lng': -76.802101, 'name': 'Maryland'},
    'MA': {'lat': 42.230171, 'lng': -71.530106, 'name': 'Massachusetts'},
    'MI': {'lat': 43.326618, 'lng': -84.536095, 'name': 'Michigan'},
    'MN': {'lat': 45.694454, 'lng': -93.900192, 'name': 'Minnesota'},
    'MS': {'lat': 32.741646, 'lng': -89.678696, 'name': 'Mississippi'},
    'MO': {'lat': 38.456085, 'lng': -92.288368, 'name': 'Missouri'},
    'MT': {'lat': 46.921925, 'lng': -110.454353, 'name': 'Montana'},
    'NE': {'lat': 41.125370, 'lng': -98.268082, 'name': 'Nebraska'},
    'NV': {'lat': 38.313515, 'lng': -117.055374, 'name': 'Nevada'},
    'NH': {'lat': 43.452492, 'lng': -71.563896, 'name': 'New Hampshire'},
    'NJ': {'lat': 40.298904, 'lng': -74.521011, 'name': 'New Jersey'},
    'NM': {'lat': 34.840515, 'lng': -106.248482, 'name': 'New Mexico'},
    'NY': {'lat': 42.165726, 'lng': -74.948051, 'name': 'New York'},
    'NC': {'lat': 35.630066, 'lng': -79.806419, 'name': 'North Carolina'},
    'ND': {'lat': 47.528912, 'lng': -99.784012, 'name': 'North Dakota'},
    'OH': {'lat': 40.388783, 'lng': -82.764915, 'name': 'Ohio'},
    'OK': {'lat': 35.565342, 'lng': -96.928917, 'name': 'Oklahoma'},
    'OR': {'lat': 44.572021, 'lng': -122.070938, 'name': 'Oregon'},
    'PA': {'lat': 40.590752, 'lng': -77.209755, 'name': 'Pennsylvania'},
    'RI': {'lat': 41.680893, 'lng': -71.511780, 'name': 'Rhode Island'},
    'SC': {'lat': 33.856892, 'lng': -80.945007, 'name': 'South Carolina'},
    'SD': {'lat': 44.299782, 'lng': -99.438828, 'name': 'South Dakota'},
    'TN': {'lat': 35.747845, 'lng': -86.692345, 'name': 'Tennessee'},
    'TX': {'lat': 31.054487, 'lng': -97.563461, 'name': 'Texas'},
    'UT': {'lat': 40.150032, 'lng': -111.862434, 'name': 'Utah'},
    'VT': {'lat': 44.045876, 'lng': -72.710686, 'name': 'Vermont'},
    'VA': {'lat': 37.769337, 'lng': -78.169968, 'name': 'Virginia'},
    'WA': {'lat': 47.400902, 'lng': -121.490494, 'name': 'Washington'},
    'WV': {'lat': 38.491226, 'lng': -80.954453, 'name': 'West Virginia'},
    'WI': {'lat': 44.268543, 'lng': -89.616508, 'name': 'Wisconsin'},
    'WY': {'lat': 42.755966, 'lng': -107.302490, 'name': 'Wyoming'},
    'DC': {'lat': 38.897438, 'lng': -77.026817, 'name': 'District of Columbia'},
    'PR': {'lat': 18.220833, 'lng': -66.590149, 'name': 'Puerto Rico'},
    'VI': {'lat': 18.335765, 'lng': -64.896335, 'name': 'Virgin Islands'},
    'GU': {'lat': 13.444304, 'lng': 144.793731, 'name': 'Guam'}
}

print(f"State coordinate database loaded: {len(STATE_COORDS)} states/territories")

In [ ]:
# Cell 7: Export State Summary JSON
print("\n" + "=" * 60)
print("EXPORTING STATE SUMMARY")
print("=" * 60)

# Enrich state summary with coordinates
state_data = []
for _, row in state_summary.iterrows():
    state_code = row['state']
    coords = STATE_COORDS.get(state_code, {'lat': 0, 'lng': 0, 'name': state_code})
    
    state_entry = {
        'state': state_code,
        'name': coords['name'],
        'lat': coords['lat'],
        'lng': coords['lng'],
        'hospital_count': int(row['hospital_count'])
    }
    
    if 'avg_readmission_rate' in row:
        state_entry['avg_readmission_rate'] = round(float(row['avg_readmission_rate']), 2)
    else:
        state_entry['avg_readmission_rate'] = 0
        
    state_entry['total_penalty_estimate'] = round(float(row.get('total_penalty_estimate', 0)), 0)
    
    state_data.append(state_entry)

# Export to JSON
with open(OUTPUT_DIR / 'state_summary.json', 'w') as f:
    json.dump(state_data, f, indent=2)

print(f"\nExported state_summary.json ({len(state_data)} states)")

In [ ]:
# Cell 8: Hospital-Level Export
print("\n" + "=" * 60)
print("HOSPITAL-LEVEL EXPORT")
print("=" * 60)

# Select top 500 hospitals by readmission rate for dashboard
if 'readmission_rate' in df_hosp.columns:
    df_hosp_clean = df_hosp.dropna(subset=['readmission_rate'])
    df_top_hospitals = df_hosp_clean.nlargest(500, 'readmission_rate')
else:
    df_top_hospitals = df_hosp.head(500)

# Prepare export
hospital_data = []
for _, row in df_top_hospitals.iterrows():
    hospital_entry = {
        'name': str(row.get('hospital_name', 'Unknown')),
        'state': str(row.get('state', 'Unknown')),
        'city': str(row.get('city', 'Unknown')) if pd.notna(row.get('city')) else 'Unknown',
        'readmission_rate': float(row.get('readmission_rate', 0)) if pd.notna(row.get('readmission_rate')) else 0,
        'penalty_pct': float(row.get('penalty_pct', 0)) if pd.notna(row.get('penalty_pct')) else 0
    }
    hospital_data.append(hospital_entry)

# Export to JSON
with open(OUTPUT_DIR / 'hospital_metrics.json', 'w') as f:
    json.dump(hospital_data, f, indent=2)

print(f"Exported hospital_metrics.json ({len(hospital_data)} hospitals)")

In [ ]:
# Cell 9: Verify All Exports
print("\n" + "=" * 60)
print("VERIFICATION OF ALL EXPORTED FILES")
print("=" * 60)

export_files = [
    OUTPUT_DIR / 'patient_risks.json',
    OUTPUT_DIR / 'risk_summary.json', 
    OUTPUT_DIR / 'state_summary.json',
    OUTPUT_DIR / 'hospital_metrics.json'
]

all_present = True
for filepath in export_files:
    if filepath.exists():
        with open(filepath, 'r') as f:
            data = json.load(f)
        if isinstance(data, list):
            print(f"  {filepath.name}: {len(data)} records")
        else:
            print(f"  {filepath.name}: {len(data.keys())} keys")
    else:
        print(f"  {filepath.name}: FILE NOT FOUND")
        all_present = False

if all_present:
    print("\n*** ALL DATA READY FOR DASHBOARD ***")
else:
    print("\n*** WARNING: Some files missing - run notebook 02 first ***")

In [ ]:
# Cell 10: Geographic Insights Summary
print("\n" + "=" * 60)
print("GEOGRAPHIC INSIGHTS SUMMARY")
print("=" * 60)

if 'avg_readmission_rate' in state_summary.columns:
    top_states = state_summary.nlargest(5, 'avg_readmission_rate')[['state', 'avg_readmission_rate', 'hospital_count']]
    bottom_states = state_summary.nsmallest(5, 'avg_readmission_rate')[['state', 'avg_readmission_rate', 'hospital_count']]
    
    print("\nHighest Readmission Rate States:")
    for _, row in top_states.iterrows():
        state_name = STATE_COORDS.get(row['state'], {}).get('name', row['state'])
        print(f"  {state_name}: {row['avg_readmission_rate']:.1f}% ({row['hospital_count']} hospitals)")
    
    print("\nLowest Readmission Rate States:")
    for _, row in bottom_states.iterrows():
        state_name = STATE_COORDS.get(row['state'], {}).get('name', row['state'])
        print(f"  {state_name}: {row['avg_readmission_rate']:.1f}% ({row['hospital_count']} hospitals)")

print("\n" + "=" * 60)
print("NEXT STEPS")
print("=" * 60)
print("""
1. Copy JSON files to dashboard:
   cp ../data/processed/*.json ../dashboard/lib/

2. Build Next.js dashboard:
   cd ../dashboard
   npm run dev

3. Deploy to Vercel:
   vercel --prod
""")